In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
import robot as Robot
import  time

robot_Instance= Robot.Robot_Class(6000,1000,90)

In [6]:
Start=time.time()
while((robot_Instance.Wait_For_A_Message(0.2))!=None):
    pass
robot_Instance.Set_Max_Try_Single_Goal(10)
robot_Instance.Set_Robot_Position(0,0,0,True)
robot_Instance.Clear_Read_Buffer()
robot_Instance.Go_To(2000,0,0)
print(time.time()-Start)

Going to 2000   0 with R 0 level 0
('G0 X2000 Y0 T3200 R0\r\n', 'M201 H0 S750\r\n', 'M3 H3\r\n')
{'g': 0, 'e': 0, 'i': 0, 'j': 0, 'a': 0, 'c': 0, 'l': 0, 'h': 0, 'd': 0, 'm': 1, 'f': 0, 'k': 0, 'b': 0}
 M0 X108.74 Y-0.18 A0.01 T2 S4096

Blocked
{'FrontRight': 1, 'FrontLeft': 1, 'Back': 1, 'Right': 1, 'BackRight': 1, 'Front': 0, 'Left': 1, 'BackLeft': 1}
Front go to left 108.68764012270594 299.81999543073874
Going to -191.25999543073874   -0.2323598772940345 with R 1 level 1
('G0 X-191.0 Y-0.0 T1000 R1\r\n', 'M201 H0 S750\r\n', 'M3 H3\r\n')
{'g': 0, 'e': 0, 'i': 0, 'j': 0, 'a': 0, 'c': 0, 'l': 0, 'h': 0, 'd': 0, 'm': 0, 'f': 0, 'k': 0, 'b': 0}
 M0 X22.84 Y8.79 A-0.20 T1 S0

Timeout
End of 1
Going to 108.68764012270594   299.81999543073874 with R 0 level 1
('G0 X109.0 Y300.0 T1000 R0\r\n', 'M201 H0 S750\r\n', 'M3 H3\r\n')
{'g': 0, 'e': 0, 'i': 0, 'j': 0, 'a': 0, 'c': 0, 'l': 0, 'h': 0, 'd': 0, 'm': 0, 'f': 0, 'k': 0, 'b': 0}
 M0 X111.61 Y103.11 A70.93 T1 S0

Timeout
End of 1
('G0 X2000 Y

In [4]:
robot_Instance.Wait_For_A_Message(0.2)